the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [2]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/new/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_valid = np.array(h['valid'])
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])
        X_train, y_train = shuffle(X_train, y_train)
        y_train = one_hot_encode(y_train)
        X_valid, y_valid = shuffle(X_valid, y_valid)
        y_valid = one_hot_encode(y_valid)
        return X_train, y_train, X_valid, y_valid
    
from keras.models import *
from keras.layers import *

def make_model(input_shape):

#     input_tensor = Input(shape=input_shape)
#     x = Dropout(0.5)(input_tensor)
#     x = Dense(10, activation='softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     input_tensor = Input(input_shape)
#     x = input_tensor
#     x = Dropout(0.5)(x)
#     x = Dense(10, activation='softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    
    x = Dense(10)(x)
    x = Activation('softmax')(x)
    model = Model(input_tensor, x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model
print("Done")

Using TensorFlow backend.


Done


ResNet50

In [3]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_ResNet50.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid))

load data: bottleneck_ResNet50.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 1.2249 - acc: 0.6450 - val_loss: 1.5959 - val_acc: 0.4178
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.46

Xception

In [4]:
X_train, y_train, X_vaiid, y_valid = load_data("bottleneck_Xception.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid))

load data: bottleneck_Xception.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 1.4959 - acc: 0.6299 - val_loss: 7.5346 - val_acc: 0.0861
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.84

InceptionV3

In [5]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_InceptionV3.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid))

load data: bottleneck_InceptionV3.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 1.4496 - acc: 0.5936 - val_loss: 1.5818 - val_acc: 0.4918
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0

VGG16

In [6]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid))

load data: bottleneck_VGG16.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 2.7655 - acc: 0.4084 - val_loss: 4.3548 - val_acc: 0.3390
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.7047 - acc: 0.7803 - val_loss: 3.8087 - val_acc: 0.3794

VGG19

In [7]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG19.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid))

load data: bottleneck_VGG19.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 3.0028 - acc: 0.3781 - val_loss: 4.8578 - val_acc: 0.2297
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.7408 - acc: 0.7639 - val_loss: 3.7642 - val_acc: 0.3274

combine  ResNet50,Xception and InceptionV3

In [8]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(2017)

X_train = []
X_valid = []

#premodels = ["bottleneck_ResNet50.h5", "bottleneck_Xception.h5", "bottleneck_InceptionV3.h5", "bottleneck_VGG16.h5", "bottleneck_VGG19.h5"]
premodels = ["bottleneck_ResNet50.h5", "bottleneck_InceptionV3.h5",  "bottleneck_VGG19.h5"]
for filename in premodels:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

------------------bottleneck_ResNet50.h5
------------------bottleneck_InceptionV3.h5
------------------bottleneck_VGG19.h5
(20787, 4608)
(20787, 10)
(1637, 4608)
(1637, 10)


In [9]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_valid,y_valid))
model.save("models/model.h5")
print("model save successed")

Train on 20787 samples, validate on 1637 samples
Epoch 1/50
20787/20787 [==============================] - 0s - loss: 0.9564 - acc: 0.7389 - val_loss: 1.5998 - val_acc: 0.4533

In [10]:
test_premodels = ["bottleneck_ResNet50_test.h5", "bottleneck_Xception_test.h5", "bottleneck_InceptionV3_test.h5", 
             "bottleneck_VGG16_test.h5", "bottleneck_VGG19_test.h5"]

test_premodels = ["bottleneck_ResNet50_test.h5", "bottleneck_InceptionV3_test.h5", "bottleneck_VGG19_test.h5"]

X_test = []
for filename in test_premodels:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_test.append(np.array(h['test']))
        
X_test = np.concatenate(X_test, axis=1)

print(X_test.shape)
assert(X_test.shape[1] == X_train.shape[1])

------------------bottleneck_ResNet50_test.h5
------------------bottleneck_InceptionV3_test.h5
------------------bottleneck_VGG19_test.h5
(79726, 4608)


In [14]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
print()
print(y_pred[:1])

import pandas as pd
from keras.preprocessing.image import *

dir = "/ext/Data/distracted_driver_detection/"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(dir + "test/", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    l.append( [name, *y_pred[i]] )
    #index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    #df.set_value(index-1, 'label', y_pred[i])
    #df.append(df2, ignore_index=True
l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df.head(10)
df = df.sort_values(by='img')
df.to_csv('pred.csv', index=None, float_format='%.3f')

79360/79726 [============================>.] - ETA: 0s 

IndexError: index 11 is out of bounds for axis 1 with size 11